In [1]:
import numpy as np
import pandas as pd
import copy

In [264]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import keras.backend as K


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read Input



## Read embedding


In [253]:
w2v = pd.read_csv('/content/drive/MyDrive/Colab/NLP/Word-Similarity/word2vec/e300.csv')
embedding = w2v.iloc[:, : 300].to_numpy()
embedding = dict(zip(w2v['word'],embedding))

## Read test

In [254]:
make_unique = {}
def readTest():
  vicon400X_noun = []
  vicon400Y_noun = []
  word_pair = []
  word_pair_noun = []
  word_pair_verb = []
  word_pair_adj = []
  with open('/content/drive/MyDrive/Colab/NLP/Word-Similarity/datasets/ViCon-400/400_noun_pairs.txt') as f:
      line = f.readline()
      for i in range(400):
          line = f.readline()
          es = line.split()
          if len(es) > 2:
              vicon400X_noun.append(np.concatenate([embedding[es[0]],embedding[es[1]]]))
              make_unique[es[0]+es[1]] = 1
              word_pair_noun.append((es[0],es[1]))
              if es[2][0]=='A':
                  vicon400Y_noun.append(0)
              else :
                  vicon400Y_noun.append(1)
        
  vicon400X_verb = []
  vicon400Y_verb = []
  with open('/content/drive/MyDrive/Colab/NLP/Word-Similarity/datasets/ViCon-400/400_verb_pairs.txt') as f:
      line = f.readline()
      for i in range(400):
          line = f.readline()
          es = line.split()
          if len(es) > 2:
              make_unique[es[0]+es[1]] = 1
              vicon400X_verb.append(np.concatenate([embedding[es[0]],embedding[es[1]]]))
              word_pair_verb.append((es[0],es[1]))
              if es[2][0]=='A':
                  vicon400Y_verb.append(0)
              else :
                  vicon400Y_verb.append(1)

  vicon400X_adj = []
  vicon400Y_adj = []
  with open('/content/drive/MyDrive/Colab/NLP/Word-Similarity/datasets/ViCon-400/600_adj_pairs.txt') as f:
      line = f.readline()
      for i in range(600):
          line = f.readline()
          es = line.split()
          if len(es) > 2:
              make_unique[es[0]+es[1]] = 1
              vicon400X_adj.append(np.concatenate([embedding[es[0]],embedding[es[1]]]))
              word_pair_adj.append((es[0],es[1]))
              if es[2][0]=='A':
                  vicon400Y_adj.append(0)
              else :
                  vicon400Y_adj.append(1)

  vicon400X = vicon400X_noun + vicon400X_verb + vicon400X_adj
  vicon400Y = vicon400Y_noun + vicon400Y_verb + vicon400Y_adj

  vicon400X_noun = np.array(vicon400X_noun)
  vicon400Y_noun = np.array(vicon400Y_noun)
  
  vicon400X_verb = np.array(vicon400X_verb)
  vicon400Y_verb = np.array(vicon400Y_verb)
  
  vicon400X_adj = np.array(vicon400X_adj)
  vicon400Y_adj = np.array(vicon400Y_adj)

  vicon400X = np.array(vicon400X)
  vicon400Y = np.array(vicon400Y)

  return vicon400X_noun,vicon400Y_noun,vicon400X_verb,vicon400Y_verb,vicon400X_adj,vicon400Y_adj , vicon400X,vicon400Y , word_pair_noun,word_pair_verb,word_pair_adj

vicon400X_noun,vicon400Y_noun,vicon400X_verb,vicon400Y_verb,vicon400X_adj,vicon400Y_adj , vicon400X,vicon400Y ,  word_pair_noun,word_pair_verb,word_pair_adj = readTest()

## Read Train
Có  2 tùy chọn:
> *isFull* để đọc dữ liệu sau khi đã dùng phương pháp Oversampling  
> *isContaintTest* để loại bỏ những dữ liệu trong tập *train* đã có trong tập *test*

In [255]:
def readInput(isFull=False, isContaintTest=True):
  path_a = "/content/drive/MyDrive/Colab/NLP/Word-Similarity/antonym-synonym set/Antonym_vietnamese.txt"
  path_s = "/content/drive/MyDrive/Colab/NLP/Word-Similarity/antonym-synonym set/Synonym_vietnamese.txt"
  if isFull:
      path_a = "/content/drive/MyDrive/Colab/NLP/Word-Similarity/antonym-synonym set/Antonym_vietnamese_full.txt"
      path_s = "/content/drive/MyDrive/Colab/NLP/Word-Similarity/antonym-synonym set/Synonym_vietnamese_full.txt"
  X = []
  Y = []
  with open(path_a) as f:
      N = 2000
      if isFull:
        N = 5921
      for i in range(N):
          line = f.readline()
          elements = line.split()
          if(elements[0]==elements[1]):
            continue
          if elements[0] in embedding and elements[1] in embedding and elements[0]+elements[1] not in make_unique:
              X.append(np.concatenate([embedding[elements[0]],embedding[elements[1]]]))
              Y.append(0)

  with open(path_s) as f:
      N = 11526
      if isFull:
          N = 51543
      for i in range(N):
          line = f.readline()
          elements = line.split()
          if len(elements) < 2 or elements[0]==elements[1]:
            continue
          if elements[0] in embedding and elements[1] in embedding and elements[0]+elements[1] not in make_unique:
              X.append(np.concatenate([embedding[elements[0]],embedding[elements[1]]]))
              Y.append(1)
  X = np.array(X)
  Y = np.array(Y)
  return X,Y

X,Y = readInput(isFull=True,isContaintTest = False)

## Split train


In [196]:
X_train, X_test, y_train, y_test = train_test_split(
       X, Y, test_size=0.2, random_state=42)

# Model


In [266]:
# f1 score
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [215]:
model = tf.keras.Sequential([
                               tf.keras.layers.Dense(100, activation = 'relu'),
                               tf.keras.layers.Dense(10, activation = 'relu'),
                               tf.keras.layers.Dense(1, activation = 'sigmoid')
                              ])
model.compile( loss= tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),
                metrics = [get_f1])

In [265]:
# model.fit(X_train,y_train)
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f' Model loss on the test set: {loss}')
# print(f' Model accuracy on the test set: {100*accuracy}')

In [202]:
#model.summary()

# Train and evaluate
 


In [219]:
model = tf.keras.Sequential([
                               tf.keras.layers.Dense(100, activation = 'relu'),
                               tf.keras.layers.Dense(10, activation = 'relu'),
                               tf.keras.layers.Dense(1, activation = 'sigmoid')
                              ])
model.compile( loss= tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),
                metrics = [get_f1])
model.fit(X, Y, epochs = 35, verbose = True)

Epoch 1/35
1327/1327 [==============================] - 5s 3ms/step - loss: 0.2593 - get_f1: 0.9413
Epoch 2/35
1327/1327 [==============================] - 5s 3ms/step - loss: 0.1591 - get_f1: 0.9632
Epoch 3/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.1142 - get_f1: 0.9740
Epoch 4/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.0913 - get_f1: 0.9790
Epoch 5/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.0755 - get_f1: 0.9825
Epoch 6/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.0629 - get_f1: 0.9848
Epoch 7/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.0580 - get_f1: 0.9858
Epoch 8/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.0482 - get_f1: 0.9883
Epoch 9/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.0456 - get_f1: 0.9889
Epoch 10/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.0454 - get_f1: 0.9894

In [251]:
model.summary()

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_112 (Dense)           (None, 100)               60100     
                                                                 
 dense_113 (Dense)           (None, 10)                1010      
                                                                 
 dense_114 (Dense)           (None, 1)                 11        
                                                                 
Total params: 61,121
Trainable params: 61,121
Non-trainable params: 0
_________________________________________________________________


In [273]:
# save model
#model.save("/content/drive/MyDrive/Colab/NLP/Word-Similarity/model_oversampling_data_300_dim")

In [272]:
# load model
#model = keras.models.load_model("/content/drive/MyDrive/Colab/NLP/Word-Similarity/model_oversampling_data_300_dim",custom_objects={'get_f1':get_f1})

In [271]:
def evaluate(vicon400X, vicon400Y,model):
  loss, accuracy = model.evaluate(vicon400X, vicon400Y)
  # print(f' Model loss on the test set: {loss}')
  print(f'Model accuracy on the test set: {100*accuracy}')
  print('-'* 100)
  y_pred = model.predict(vicon400X)
  y_pred_pro = copy.deepcopy(y_pred)
  y_pred[y_pred>0.5] = 1
  y_pred[y_pred<=0.5] = 0
  print('Confusion matix :\n')
  print(confusion_matrix(vicon400Y, y_pred))

  print('-' * 100) 
  precision, recall, fscore, support = score(vicon400Y, y_pred,average='macro')
  print('precision: {}'.format(precision))
  print('recall: {}'.format(recall))
  print('f1score: {}'.format(fscore))
  # print('support: {}'.format(support))
evaluate(vicon400X, vicon400Y,model)

44/44 [==============================] - 0s 3ms/step - loss: 1.2357 - get_f1: 0.8768
Model accuracy on the test set: 87.68333792686462
----------------------------------------------------------------------------------------------------
Confusion matix :

[[535 165]
 [ 26 674]]
----------------------------------------------------------------------------------------------------
precision: 0.8784957476326754
recall: 0.8635714285714285
f1score: 0.8622131738427632


In [259]:
# đánh giá kết quả trên các loại từ 
def evaluate_each_type(vicon400X, vicon400Y , model , type):     
  loss, accuracy = model.evaluate(vicon400X, vicon400Y)
  # print(f' Model loss on the test set: {loss}')
  print(f'Type: {type}')  
  print('-'* 100)
  y_pred = model.predict(vicon400X)
  y_pred_pro = copy.deepcopy(y_pred)
  y_pred[y_pred>0.5] = 1
  y_pred[y_pred<=0.5] = 0
  word_pair = []
  if type =='noun' :
      word_pair = word_pair_noun
  elif type == 'verb':
      word_pair = word_pair_verb
  else :
      word_pair = word_pair_adj

  precision, recall, fscore, support = precision_recall_fscore_support(vicon400Y, y_pred,average='macro')
  print('precision: {}'.format(precision))
  print('recall: {}'.format(recall))
  print('f1score: {}'.format(fscore))

  # debug step
  print('-'*100)
  print('wrong prediction:')
  diff = vicon400Y - y_pred.reshape(-1)
  tword_pair = np.array(word_pair)
  vicon400Y=vicon400Y.reshape(len(vicon400Y),1)
  debug_arr = np.concatenate(  (tword_pair[np.where(diff!=0)] , vicon400Y[np.where(diff!=0)]),axis=1 )
  print(debug_arr)

## Evaluate with noun

In [260]:
# evaluate with noun
evaluate_each_type(vicon400X_noun, vicon400Y_noun,model,'noun')


13/13 [==============================] - 0s 3ms/step - loss: 2.5136 - get_f1: 0.8004
Type: noun
----------------------------------------------------------------------------------------------------
precision: 0.8140046054008792
recall: 0.7625
f1score: 0.7523445799307868
----------------------------------------------------------------------------------------------------
wrong prediction:
[['khoái_lạc' 'nỗi_đau' '0']
 ['thanh_danh' 'ô_nhục' '0']
 ['ba' 'me' '0']
 ['ngày' 'ban_đêm' '0']
 ['khuyết_thiếu' 'ưu_điểm' '0']
 ['bệ' 'đầu' '0']
 ['thầy' 'u' '0']
 ['dĩ_vãng' 'tương_lai' '0']
 ['đàn_bà' 'nam_giới' '0']
 ['xuân_phân' 'thu_phân' '0']
 ['nữ_hoàng' 'vua' '0']
 ['nam' 'trai' '1']
 ['phụ_thân' 'má' '0']
 ['thiếu_nữ' 'chàng_trai' '0']
 ['sở_trường' 'nhược_điểm' '0']
 ['đàn_bà' 'nam' '0']
 ['tinh_mơ' 'hoàng_hôn' '0']
 ['tiểu_phẫu' 'đại_phẫu' '0']
 ['quá_khứ' 'vị_lai' '0']
 ['thầy' 'me' '0']
 ['dĩ_vãng' 'sau_này' '0']
 ['bố' 'má' '0']
 ['ông_già' 'u' '0']
 ['sở_trường' 'điểm_yếu' '0']
 ['tiếp

## Evaluate with verb


In [261]:
# evaluate with verb
evaluate_each_type(vicon400X_verb, vicon400Y_verb,model,'verb')


13/13 [==============================] - 0s 3ms/step - loss: 1.0865 - get_f1: 0.8964
Type: verb
----------------------------------------------------------------------------------------------------
precision: 0.8960549817504078
recall: 0.8825000000000001
f1score: 0.8814859642818869
----------------------------------------------------------------------------------------------------
wrong prediction:
[['giải_thoát' 'thi_hành' '0']
 ['kìm_giữ' 'trói_buộc' '1']
 ['lẩn_tránh' 'áp' '0']
 ['nhận' 'loại_bỏ' '0']
 ['phản_kháng' 'khuất_phục' '0']
 ['giả_mạo' 'mạo' '1']
 ['hạ' 'tăng' '0']
 ['thanh_trừng' 'phục_chức' '0']
 ['khuất_núi' 'chào_đời' '0']
 ['bán' 'sắm' '0']
 ['ngã' 'bổ' '1']
 ['phạm' 'giữ' '0']
 ['thu' 'xoá_bỏ' '0']
 ['bổ_nhậm' 'miễn_nhiệm' '0']
 ['khởi_động' 'ngừng' '0']
 ['bất_an' 'xả' '0']
 ['trôi' 'đắm' '0']
 ['chìm' 'nổi' '0']
 ['tránh_mặt' 'đối_mặt' '0']
 ['nhập' 'tách' '0']
 ['va' 'trật' '0']
 ['nằm' 'ngồi' '0']
 ['đập' 'hụt' '0']
 ['quí_mến' 'căm' '0']
 ['bán' 'mua' '0']
 ['ca_

## Evaluate with adj


In [262]:
#evaluate with adj
evaluate_each_type(vicon400X_adj, vicon400Y_adj,model,'adj')

19/19 [==============================] - 0s 2ms/step - loss: 0.4832 - get_f1: 0.9158
Type: adj
----------------------------------------------------------------------------------------------------
precision: 0.9203932603088467
recall: 0.9183333333333333
f1score: 0.9182331689653158
----------------------------------------------------------------------------------------------------
wrong prediction:
[['đen' 'đỏ' '0']
 ['gần' 'xa' '0']
 ['chặt' 'lỏng' '0']
 ['màu_mỡ' 'phì_nhiêu' '1']
 ['khô' 'khô_khan' '1']
 ['nghèo_khó' 'nghèo_túng' '1']
 ['nườm_nượp' 'vắng_ngắt' '0']
 ['lớn' 'to' '1']
 ['bạc' 'tệ' '1']
 ['dễ' 'khó' '0']
 ['tỉnh_táo' 'mê_muội' '0']
 ['nhỡ' 'vừa' '1']
 ['nhạt_thếch' 'mặn_mòi' '0']
 ['cực_khổ' 'sung_sướng' '0']
 ['hà_tiện' 'hoang' '0']
 ['thường' 'lạ_thường' '0']
 ['lai' 'thuần_chủng' '0']
 ['keo_kiệt' 'đá' '1']
 ['chặt' 'lỏng_lẻo' '0']
 ['lắm' 'ối' '1']
 ['mới_tinh' 'mới_toanh' '1']
 ['thường' 'khác_thường' '0']
 ['giầu' 'khó' '0']
 ['bí_mật' 'công_khai' '0']
 ['khác_biệt'

# Debug in test set

In [ ]:
# diff = vicon400Y - y_pred.reshape(-1)
# np.where(diff!=0)
# tword_pair = np.array(word_pair)
# tword_pair[np.where(diff!=0)]
# y_pred_pro[np.where(diff!=0)]

# Evaluate in origin data
> Đánh giá mô hình với dữ liệu gốc khi chưa dùng over-sampling


In [221]:
w2v = pd.read_csv('/content/drive/MyDrive/Colab/NLP/Word-Similarity/word2vec/e300.csv')
embedding = w2v.iloc[:, : 300].to_numpy()
embedding = dict(zip(w2v['word'],embedding))
make_unique = {}
vicon400X_noun,vicon400Y_noun,vicon400X_verb,vicon400Y_verb,vicon400X_adj,vicon400Y_adj , vicon400X,vicon400Y ,  word_pair_noun,word_pair_verb,word_pair_adj = readTest()
X,Y = readInput(isFull=False)

#print(X.shape)
def eveluate_origin_data():
  model = tf.keras.Sequential([
                               tf.keras.layers.Dense(100, activation = 'relu'),
                               tf.keras.layers.Dense(10, activation = 'relu'),
                               tf.keras.layers.Dense(1, activation = 'sigmoid')
                              ])
  model.compile( loss= tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),
                metrics = [get_f1])
  model.fit(X, Y, epochs = 35, verbose = True)
  y_pred = model.predict(vicon400X)
  y_pred_pro = copy.deepcopy(y_pred)
  y_pred[y_pred>0.5] = 1
  y_pred[y_pred<=0.5] = 0
  precision, recall, fscore, support = score(vicon400Y, y_pred , average='macro')
  print('precision: {}'.format(precision))
  print('recall: {}'.format(recall))
  print('f1score: {}'.format(fscore))
  # print('support: {}'.format(support))

eveluate_origin_data()

Epoch 1/35
360/360 [==============================] - 2s 3ms/step - loss: 0.1951 - get_f1: 0.9663
Epoch 2/35
360/360 [==============================] - 1s 4ms/step - loss: 0.1220 - get_f1: 0.9751
Epoch 3/35
360/360 [==============================] - 1s 3ms/step - loss: 0.0770 - get_f1: 0.9862
Epoch 4/35
360/360 [==============================] - 1s 3ms/step - loss: 0.0480 - get_f1: 0.9913
Epoch 5/35
360/360 [==============================] - 1s 3ms/step - loss: 0.0314 - get_f1: 0.9951
Epoch 6/35
360/360 [==============================] - 1s 3ms/step - loss: 0.0284 - get_f1: 0.9957
Epoch 7/35
360/360 [==============================] - 1s 3ms/step - loss: 0.0199 - get_f1: 0.9967
Epoch 8/35
360/360 [==============================] - 1s 3ms/step - loss: 0.0107 - get_f1: 0.9981
Epoch 9/35
360/360 [==============================] - 1s 3ms/step - loss: 0.0140 - get_f1: 0.9978
Epoch 10/35
360/360 [==============================] - 1s 3ms/step - loss: 0.0121 - get_f1: 0.9979
Epoch 11/35
360/360

# Evalue in 50 dimensions embedding


In [242]:
w2v = pd.read_csv('/content/drive/MyDrive/Colab/NLP/Word-Similarity/word2vec/e50.csv')
embedding = w2v.iloc[:, : 50].to_numpy()
embedding = dict(zip(w2v['word'],embedding))
make_unique = {}
vicon400X_noun,vicon400Y_noun,vicon400X_verb,vicon400Y_verb,vicon400X_adj,vicon400Y_adj , vicon400X,vicon400Y , word_pair_noun,word_pair_verb,word_pair_adj = readTest()
X,Y = readInput(isFull=True)
def eveluate_in_50_dimensions_data():
  model = tf.keras.Sequential([
                               tf.keras.layers.Dense(100, activation = 'relu'),
                               tf.keras.layers.Dense(10, activation = 'relu'),
                               tf.keras.layers.Dense(1, activation = 'sigmoid')
                              ])
  model.compile( loss= tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),
                metrics = [get_f1])
  model.fit(X, Y, epochs = 35, verbose = True)
  y_pred = model.predict(vicon400X)
  y_pred_pro = copy.deepcopy(y_pred)
  y_pred[y_pred>0.5] = 1
  y_pred[y_pred<=0.5] = 0
  precision, recall, fscore, support = score(vicon400Y, y_pred , average='macro')
  print('precision: {}'.format(precision))
  print('recall: {}'.format(recall))
  print('f1score: {}'.format(fscore))
  # print('support: {}'.format(support))

eveluate_in_50_dimensions_data()

Epoch 1/35
1327/1327 [==============================] - 7s 4ms/step - loss: 0.2795 - get_f1: 0.9400
Epoch 2/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.2142 - get_f1: 0.9521
Epoch 3/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.1844 - get_f1: 0.9588
Epoch 4/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.1658 - get_f1: 0.9620
Epoch 5/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.1498 - get_f1: 0.9651
Epoch 6/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.1344 - get_f1: 0.9684
Epoch 7/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.1251 - get_f1: 0.9712
Epoch 8/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.1136 - get_f1: 0.9741
Epoch 9/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.1081 - get_f1: 0.9747
Epoch 10/35
1327/1327 [==============================] - 4s 3ms/step - loss: 0.1023 - get_f1: 0.9761